In [21]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt

In [22]:
input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 3 # number of times which the entire dataset is passed throughout the model
batch_size = 64  # the size of input data took for one iteration
lr = 5e-3 # size of step

In [23]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [24]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [25]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [26]:
net = Net(input_size, hidden_size, num_classes)

In [27]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [28]:
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28))
    labels = Variable(labels)
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Epoch [1/3], Step [100/937], Loss: 0.8648
Epoch [1/3], Step [200/937], Loss: 0.0988
Epoch [1/3], Step [300/937], Loss: 0.1235
Epoch [1/3], Step [400/937], Loss: 0.2097
Epoch [1/3], Step [500/937], Loss: 0.2690
Epoch [1/3], Step [600/937], Loss: 0.0646
Epoch [1/3], Step [700/937], Loss: 0.0333
Epoch [1/3], Step [800/937], Loss: 0.1393
Epoch [1/3], Step [900/937], Loss: 0.0946
Epoch [2/3], Step [100/937], Loss: 0.1629
Epoch [2/3], Step [200/937], Loss: 0.1339
Epoch [2/3], Step [300/937], Loss: 0.0829
Epoch [2/3], Step [400/937], Loss: 0.0610
Epoch [2/3], Step [500/937], Loss: 0.0774
Epoch [2/3], Step [600/937], Loss: 0.0384
Epoch [2/3], Step [700/937], Loss: 0.0459
Epoch [2/3], Step [800/937], Loss: 0.3028
Epoch [2/3], Step [900/937], Loss: 0.0227
Epoch [3/3], Step [100/937], Loss: 0.0593
Epoch [3/3], Step [200/937], Loss: 0.0337
Epoch [3/3], Step [300/937], Loss: 0.0089
Epoch [3/3], Step [400/937], Loss: 0.0091
Epoch [3/3], Step [500/937], Loss: 0.0845
Epoch [3/3], Step [600/937], Loss:

In [29]:
correct = 0
total = 0

for images,labels in train_gen:
  images = images.view(-1,28*28)
  labels = labels
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)
train_acc = (100*correct.numpy())/(total+1)
print('Train accuracy of the model: %.3f %%' %(train_acc))
print(correct, total)

correct = 0
total = 0
for images,labels in test_gen:
  images = images.view(-1,28*28)
  labels = labels
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)
test_acc = (100*correct.numpy())/(total+1)
print('Test accuracy of the model: %.3f %%' %(test_acc))
print(correct, total)

Train accuracy of the model: 98.297 %
tensor(58979) 60000
Test accuracy of the model: 97.170 %
tensor(9718) 10000
